In [1]:
let x = "Hello World" in print x

"Hello World"

In [2]:
import Data.ByteString.Lazy as BS
import qualified Data.Csv.HMatrix as CSV
import Numeric.LinearAlgebra
import Data.Csv (HasHeader (..))
import Numeric.LinearAlgebra.Data
import Prelude hiding ((<>))
import Data.Function ((&))

In [3]:
variance :: Matrix Double -> Matrix Double
variance = asRow . takeDiag . unSym . snd . meanCov

-- | Principal components analysis.
-- Expects normalised matrix as an input value.
--
-- The covariance matrix of z-scores (normalised matrix) is equal to the Pearson correlation coefficient matrix.
-- This function expects normalised matrix as its input, so taking its covariance gives us correlation
-- that can be used to find eigenvectors.
pca :: Matrix Double -> Matrix Double
pca m = m <> (snd . eigSH . snd . meanCov) m  

-- | Explains variance ratio between "original" normalised matrix and PCA matrix
explainPcaRatio :: Matrix Double -> Matrix Double -> Double
explainPcaRatio mnorm mpca =
  let pcaVar = variance mpca
      normVar = variance mnorm
  in sumElements pcaVar / sumElements normVar * 100

In [4]:
wine_csv <- BS.readFile "./wine-quality/winequality-red.csv"
raw_data = CSV.decodeMatrixWith HasHeader ';' wine_csv

In [5]:
(rawMean, rawCov) = meanCov raw_data

-- variance is a diagonal of a covariance matrix
rawVariance = unSym rawCov & takeDiag & asRow
rawStdDev = sqrt rawVariance

In [6]:
centered = raw_data - asRow rawMean
normData = centered / rawStdDev

In [7]:
normInput  = normData ?? (All, DropLast 1)
normOutput = normData ?? (All, TakeLast 1)

In [8]:
components = pca normInput
dispShort 5 12 4 components

1599x11
1.6190  -0.4508  -1.7739   0.0437   0.0670   0.9136  -0.1610   0.2822   0.0051   0.2677  -0.0486
0.7989  -1.8560  -0.9114   0.5479  -0.0184  -0.9294  -1.0095  -0.7623  -0.5205  -0.0628   0.1381
 :        :        :        :        :        :        :        :        :        :        :    
2.2698  -0.9795   0.6278   0.6396   0.0677   0.8601  -0.3214   0.4687  -0.6121  -0.7792  -0.0409
0.4268   0.5365   1.6284  -0.3916   0.4503   0.4960   1.1888  -0.0422   0.4042  -0.7792   0.4496

In [9]:
explainPcaRatio normInput (components ¿ [0..3])

70.80743772386744